<a href="https://colab.research.google.com/github/Sarnapa/Credit-Card-Fraud-Detection/blob/main/Notebooks/CreditCardFraudDetection_AnomalyDetectionAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---- Załadowanie danych


# Załadowanie danych


# Analiza danych

# Under / oversampling

# K krotna walidacja

# Miary jakości

# Metody nadzorowane

# Metody nienadzorowane